## 1. Импорт библиотек и настройка параметров

In [1]:
# Импортируем библиотеки:

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import time
import os
import re

In [2]:
# Увеличиваем число отображаемых строк и столбцов в pandas:

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [3]:
# Задаем дирректорию для работы с файлами:

# При выполнении на kaggle раскомментировать этот блок:
# for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
# PATH_to_file = '/kaggle/input/sf-dst-scoring/'

# При выполнении локально раскомментировать этот блок:
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
PATH_to_file = './data/'

./data/cars_dicts.txt
./data/cars_dicts_forward.txt
./data/.DS_Store
./data/not_read_links_revers.txt
./data/car_links.txt
./data/test.csv
./data/cars_dicts_revers.txt
./data/train.csv
./data/test_finished.csv
./data/cars_dicts_temp.txt
./data/sample_submission.csv
./data/not_read_links_forward.txt


## 2. Загрузка тренировочного и тестового датасетов

### 2.1 Загрузка данных

In [4]:
# Загрузим наш предобаботанный тестовый датасет:

test = pd.read_csv(PATH_to_file + 'test_finished.csv')

print('Размер тестового датасета: ', test.shape)
display(test.sample(5))

Размер тестового датасета:  (34686, 21)


,body_type,brand,color,description,engine_volume,engine_power,equipment,fuel_type,mileage,model_date,model_name,doors_num,parsing_unixtime,production_date,transmission,vendor,owners_count,ownership,docs,drive,steering
12683,внедорожник 5 дв.,NISSAN,серебристый,"Полная комплектация, 7 мест, турботаймер, 2 ко...",2.5 LTR,190 N12,"{""cruise-control"":true,""asr"":true,""tinted-glas...",дизель,230000,2010,PATHFINDER,5,1603275645,2011,автоматическая,JAPANESE,2,NaN,Оригинал,полный,Левый
11993,седан,NISSAN,красный,Перекуп мимо! Прошу Вас не терять время и не з...,1.8 LTR,116 N12,"{""engine-proof"":true,""airbag-driver"":true,""aux...",бензин,160000,2002,ALMERA,4,1603272949,2005,автоматическая,JAPANESE,3,NaN,Оригинал,передний,Левый
31614,седан,MERCEDES,чёрный,Куплена у официального дилера в сентябре 2014 ...,1.6 LTR,156 N12,"{""navigation"":true,""sport-suspension"":true}",бензин,93500,2014,C_KLASSE,4,1603588228,2014,автоматическая,EUROPEAN,1,73.0,Оригинал,задний,Левый
29317,хэтчбек 3 дв.,BMW,красный,"Бьется 2 владельца, по факту 1 (с жены на мужа...",1.6 LTR,115 N12,"{""alloy-wheel-disks"":true,""condition"":true,""co...",бензин,102000,2007,1ER,3,1603559675,2010,автоматическая,EUROPEAN,1,NaN,Оригинал,задний,Левый
33502,внедорожник 5 дв.,TOYOTA,чёрный,Автомобиль в моем пользовании больше трех л...,4.7 LTR,235 N12,"{""asr"":true,""roller-blind-for-rear-window"":tru...",бензин,438000,2002,LAND_CRUISER,5,1603616795,2003,автоматическая,JAPANESE,3,NaN,Оригинал,полный,Левый


In [5]:
# Загрузим наш предобаботанный тренировочный датасет:

train = pd.read_csv(PATH_to_file + 'train.csv')

print('Размер тестового датасета: ', train.shape)
display(train.sample(5))

Размер тестового датасета:  (908, 22)


,body_type,brand,color,description,engine_volume,engine_power,equipment,fuel_type,mileage,model_date,model_name,doors_num,parsing_unixtime,production_date,transmission,vendor,owners_count,ownership,docs,drive,steering,price
808,внедорожник 5 дв.,SUBARU,чёрный,ЛОТ: 01271936\nFAVORIT MOTORS KIA Восток\n\n🎯 ...,2.5 LTR,171 N12,"{'cruise-control': True, 'asr': True, 'tinted-...",бензин,143252,2012,FORESTER,5,1631024923,2013,вариатор,JAPANESE,1,NaN,ORIGINAL,ALL_WHEEL_DRIVE,LEFT,1199000
673,внедорожник 5 дв.,BMW,чёрный,"Автомобиль в отличном техническом состоянии, н...",2.0 LTR,177 N12,"{'cruise-control': True, 'asr': True, 'airbag-...",дизель,137000,2009,X1,5,1631023584,2012,автоматическая,EUROPEAN,1,NaN,ORIGINAL,ALL_WHEEL_DRIVE,LEFT,999000
702,универсал 5 дв.,VOLVO,серый,Мы предлагаем Вам сформированные индивидуальны...,2.0 LTR,190 N12,"{'asr': True, 'tinted-glass': True, 'esp': Tru...",дизель,55396,2016,V90_CROSS_COUNTRY,5,1631023726,2018,автоматическая,EUROPEAN,2,NaN,ORIGINAL,ALL_WHEEL_DRIVE,LEFT,3197000
823,лифтбек,RENAULT,серебристый,Торг есть.\n\nПродам свой авто. Очень просторн...,1.5 LTR,110 N12,"{'engine-proof': True, 'cruise-control': True,...",дизель,221000,2007,LAGUNA,5,1631025029,2009,механическая,EUROPEAN,4,NaN,ORIGINAL,FORWARD_CONTROL,LEFT,490000
640,внедорожник 5 дв.,INFINITI,чёрный,Официальный дилерский центр РОЛЬФ Сити | Автом...,3.0 LTR,238 N12,"{'cruise-control': True, 'asr': True, 'tinted-...",дизель,125338,2011,FX,5,1631023367,2012,автоматическая,JAPANESE,2,NaN,ORIGINAL,ALL_WHEEL_DRIVE,LEFT,1877000


### 2.1 "Синхронизация" датасетов

In [10]:
# Наши датасеты хоть и похожи, но не совсем идентичны, перед началом EDA необходимо убедиться,
# что данные в них хранятся в одинаковых форматах.

display(test.info())

display(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   body_type         34686 non-null  object 
 1   brand             34686 non-null  object 
 2   color             34686 non-null  object 
 3   description       34686 non-null  object 
 4   engine_volume     34686 non-null  object 
 5   engine_power      34686 non-null  object 
 6   equipment         24690 non-null  object 
 7   fuel_type         34686 non-null  object 
 8   mileage           34686 non-null  int64  
 9   model_date        34686 non-null  int64  
 10  model_name        34686 non-null  object 
 11  doors_num         34686 non-null  int64  
 12  parsing_unixtime  34686 non-null  int64  
 13  production_date   34686 non-null  int64  
 14  transmission      34686 non-null  object 
 15  vendor            34686 non-null  object 
 16  owners_count      34686 non-null  int64 

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   body_type         590 non-null    object 
 1   brand             590 non-null    object 
 2   color             590 non-null    object 
 3   description       590 non-null    object 
 4   engine_volume     590 non-null    object 
 5   engine_power      590 non-null    object 
 6   equipment         590 non-null    object 
 7   fuel_type         590 non-null    object 
 8   mileage           590 non-null    int64  
 9   model_date        590 non-null    int64  
 10  model_name        590 non-null    object 
 11  doors_num         590 non-null    int64  
 12  parsing_unixtime  590 non-null    int64  
 13  production_date   590 non-null    int64  
 14  transmission      590 non-null    object 
 15  vendor            590 non-null    object 
 16  owners_count      590 non-null    int64  
 1

None

In [12]:
# Названия колонок и их типы (кроме отсутствующей у тестовой выборки колонки price) совпадают.
# Теперь посмотрим на кол-во. значений в них:

display(train.nunique().sort_values())

display(test.nunique().sort_values())

docs                  2
steering              2
vendor                2
drive                 3
doors_num             4
owners_count          4
transmission          4
fuel_type             5
color                16
body_type            17
brand                24
production_date      36
model_date           41
engine_volume        45
ownership            75
engine_power        184
model_name          276
equipment           504
description         542
price               575
mileage             581
parsing_unixtime    590
dtype: int64

steering                2
docs                    2
vendor                  2
owners_count            3
drive                   3
transmission            4
fuel_type               5
doors_num               5
brand                  12
color                  16
body_type              24
engine_volume          55
model_date             66
production_date        69
ownership             244
engine_power          315
model_name            544
mileage             11268
equipment           23705
description         31732
parsing_unixtime    34686
dtype: int64

#### 2.1.1 Бинарные признаки

In [13]:
# Начнем с Бинарных переменных. Посмотрим как они закодированы в обоих датасетах:

bin_cols = ['steering', 'docs', 'vendor']

print('----- Тестовый датасет -----')
for col in bin_cols:
    print(f'Значения в колнке {col}: {test[col].unique()}')
    
print('----- Тренировочный датасет -----')
for col in bin_cols:
    print(f'Значения в колнке {col}: {train[col].unique()}')

----- Тестовый датасет -----
Значения в колнке steering: ['Левый' 'Правый']
Значения в колнке docs: ['Оригинал' 'Дубликат']
Значения в колнке vendor: ['EUROPEAN' 'JAPANESE']
----- Тренировочный датасет -----
Значения в колнке steering: ['LEFT' 'RIGHT']
Значения в колнке docs: ['ORIGINAL' 'DUPLICATE']
Значения в колнке vendor: ['EUROPEAN' 'JAPANESE']


In [17]:
# Кодировка значений в колонках steering и docs не совпадает. 
# Приведем ее к единому варианту (латинскими буквами): 


test.steering = test.steering.apply(lambda x: 'LEFT' if x == 'Левый' else 'RIGHT')
test.docs = test.docs.apply(lambda x: 'ORIGINAL' if x == 'Оригинал' else 'DUPLICATE')

#### 2.1.2 Категориаьные переменные (drive, owners_count, transmission, fuel_type, doors_num)

In [30]:
cat_cols = ['drive', 'owners_count', 'transmission', 'fuel_type', 'doors_num']

print('----- Тестовый датасет -----')
for col in cat_cols:
    print(f'Значения в колнке {col}: {test[col].unique()}')
    
print('----- Тренировочный датасет -----')
for col in cat_cols:
    print(f'Значения в колнке {col}: {train[col].unique()}')

----- Тестовый датасет -----
Значения в колнке drive: ['передний' 'полный' 'задний']
Значения в колнке owners_count: [3 1 2]
Значения в колнке transmission: ['роботизированная' 'механическая' 'автоматическая' 'вариатор']
Значения в колнке fuel_type: ['бензин' 'дизель' 'гибрид' 'электро' 'газ']
Значения в колнке doors_num: [5 4 2 3 0]
----- Тренировочный датасет -----
Значения в колнке drive: ['ALL_WHEEL_DRIVE' 'FORWARD_CONTROL' 'REAR_DRIVE']
Значения в колнке owners_count: [3 4 1 2]
Значения в колнке transmission: ['механическая' 'роботизированная' 'автоматическая' 'вариатор']
Значения в колнке fuel_type: ['бензин' 'дизель' 'гибрид' 'газ' 'электро']
Значения в колнке doors_num: [5 4 3 2]


In [38]:
# Расхождения наблюдаются в колонке drive. Кроме того в  колонке owners_count тренировочного
# датасета больше вариантов значний (в тестовом датасете 3 означает 3 и более).
# Исправим это:

train.owners_count = train.owners_count.apply(lambda x: x if x < 3 else 3)

drive_dict = {'передний' : 'FORWARD_CONTROL', 
              'полный' : 'ALL_WHEEL_DRIVE',
              'задний' : 'REAR_DRIVE'}

test.drive = test.drive.apply(lambda x: drive_dict[x])

### 2.2 Объединение датасетов

In [49]:
# Теперь подготовимм наши датасеты к объединению. К тестотовому датасету
# добавим пустую колонку price:

test['price'] = np.nan

In [50]:
# Добавим столбец test. У тестового датасета значение будет 1, у тренировочного 0:

test['test'] = 1
train['test'] = 0

In [54]:
# Теперь объединим датасеты:

df = pd.concat([test, train], ignore_index = True)

display(df.shape)

display(df.sample(10))

(35276, 23)

,body_type,brand,color,description,engine_volume,engine_power,equipment,fuel_type,mileage,model_date,model_name,doors_num,parsing_unixtime,production_date,transmission,vendor,owners_count,ownership,docs,drive,steering,price,test
10627,внедорожник 5 дв.,BMW,чёрный,Все вопросы по телефону звоните отвечу,3.0 LTR,231 N12,NaN,бензин,360000,2003,X5,5,1603119776,2004,автоматическая,european,3,NaN,original,all_wheel_drive,left,NaN,1
9158,хэтчбек 5 дв.,BMW,синий,Автодилер года 2020\n\nПобедитель в номинации ...,1.5 LTR,136 N12,"{""cruise-control"":true,""engine-proof"":true,""as...",бензин,10481,2017,1ER,5,1603114015,2019,автоматическая,european,1,NaN,original,rear_drive,left,NaN,1
1230,лифтбек,SKODA,чёрный,Автодилер года 2020\n\nПобедитель в номинации ...,1.6 LTR,110 N12,"{""airbag-driver"":true,""aux"":true,""usb"":true,""i...",бензин,64783,2017,OCTAVIA,5,1603230916,2017,автоматическая,european,2,NaN,original,forward_control,left,NaN,1
32255,внедорожник 5 дв.,MITSUBISHI,белый,Состояние автомобиля хорошее. Причина продажи ...,1.8 LTR,140 N12,"{""engine-proof"":true,""alloy-wheel-disks"":true,...",бензин,70500,2012,ASX,5,1603599006,2013,вариатор,japanese,1,81.0,original,forward_control,left,NaN,1
18327,внедорожник 5 дв.,MERCEDES,чёрный,Авто в отличном состоянии. Вложений не требует...,3.5 LTR,272 N12,NaN,бензин,310000,2005,M_KLASSE,5,1603251251,2006,автоматическая,european,3,16.0,duplicate,all_wheel_drive,left,NaN,1
11110,седан,NISSAN,серебристый,ПТС ОРИГИНАЛ !\nДВА ОСБСТВЕННИКА ЗА ВСЕ ВРЕМЯ ...,2.5 LTR,182 N12,"{""cruise-control"":true,""tinted-glass"":true,""el...",бензин,152000,2008,TEANA,4,1603269510,2008,вариатор,japanese,2,NaN,original,forward_control,left,NaN,1
32124,внедорожник 5 дв.,MITSUBISHI,серый,-Оригинальный ПТС. \n-Один владелец. \t\n-Серв...,3.0 LTR,222 N12,"{""cruise-control"":true,""asr"":true,""airbag-driv...",бензин,37948,2013,PAJERO_SPORT,5,1603598191,2015,автоматическая,japanese,1,NaN,original,all_wheel_drive,left,NaN,1
19083,внедорожник 5 дв.,TOYOTA,красный,"Автомобиль в хорошем техническом состоянии, по...",2.8 LTR,177 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",дизель,83000,2013,LAND_CRUISER_PRADO,5,1603286517,2016,автоматическая,japanese,2,41.0,original,all_wheel_drive,left,NaN,1
12554,внедорожник 5 дв.,NISSAN,чёрный,ПО ФАКТУ 1 ВЛАДЕЛЕЦ !!\nДО НАЛОГОВЫЙ БЕНЗИНОВЫ...,3.5 LTR,249 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,89000,2012,PATHFINDER,5,1603275156,2015,вариатор,japanese,2,NaN,original,all_wheel_drive,left,NaN,1
21016,хэтчбек 5 дв.,TOYOTA,серый,Машина в идеальном состоянии!Сам хозяин!\nКомп...,1.6 LTR,110 N12,"{""tinted-glass"":true,""airbag-driver"":true,""usb...",бензин,280000,2003,COROLLA,5,1603293940,2006,механическая,japanese,3,NaN,original,forward_control,left,NaN,1


## 3. Обработка общего датасета

In [55]:
# Посмотрим информацию по объединенному датасету:

display(df.info())

display(df.nunique().sort_values())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35276 entries, 0 to 35275
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   body_type         35276 non-null  object 
 1   brand             35276 non-null  object 
 2   color             35276 non-null  object 
 3   description       35276 non-null  object 
 4   engine_volume     35276 non-null  object 
 5   engine_power      35276 non-null  object 
 6   equipment         25280 non-null  object 
 7   fuel_type         35276 non-null  object 
 8   mileage           35276 non-null  int64  
 9   model_date        35276 non-null  int64  
 10  model_name        35276 non-null  object 
 11  doors_num         35276 non-null  int64  
 12  parsing_unixtime  35276 non-null  int64  
 13  production_date   35276 non-null  int64  
 14  transmission      35276 non-null  object 
 15  vendor            35276 non-null  object 
 16  owners_count      35276 non-null  int64 

None

test                    2
steering                2
docs                    2
vendor                  2
drive                   3
owners_count            3
transmission            4
doors_num               5
fuel_type               5
color                  16
brand                  24
body_type              24
engine_volume          56
model_date             67
production_date        70
ownership             258
engine_power          325
price                 575
model_name            645
mileage             11751
equipment           24209
description         32272
parsing_unixtime    35276
dtype: int64

In [69]:
# Посмотрим как закодированы данные в бинарных и категориальных признаках:

bin_cat = bin_cols
bin_cat.extend(cat_cols)

for col in bin_cat:
    print(f'Значения в колнке {col}: {df[col].unique()}')

Значения в колнке steering: ['left' 'right']
Значения в колнке docs: ['original' 'duplicate']
Значения в колнке vendor: ['european' 'japanese']
Значения в колнке drive: ['forward_control' 'all_wheel_drive' 'rear_drive']
Значения в колнке owners_count: [3 1 2]
Значения в колнке transmission: ['роботизированная' 'механическая' 'автоматическая' 'вариатор']
Значения в колнке fuel_type: ['бензин' 'дизель' 'гибрид' 'электро' 'газ']
Значения в колнке doors_num: [5 4 2 3 0]


In [70]:
# Значения в колонках transmission и fuel_type написаны по руски.
# Пересохраним их по английски:

transmission_dict = {'роботизированная' : 'robotic', 
                      'механическая' : 'mechanical',
                      'автоматическая' : 'automatic',
                      'вариатор' : 'variator'}

fuel_dict = {'бензин' : 'gasoline', 
             'дизель' : 'diesel',
             'гибрид' : 'hybrid',
             'электро' : 'electro',
             'газ' : 'gas'}

df.transmission = df.transmission.apply(lambda x: transmission_dict[x])
df.fuel_type = df.fuel_type.apply(lambda x: fuel_dict[x])

In [ ]:
# А так-же пересохраним столбцы ___________________________________ строчными буквами:

In [ ]:
# А заодно запишем все бинарные столбцы строчными буквами:

bin_cols = ['steering', 'docs', 'vendor']

for col in bin_cols:
    test[col] = test[col].apply(lambda x: x.lower())
    train[col] = train[col].apply(lambda x: x.lower())

In [ ]:
# Теперь можно перевести наши бинарные столбцы в числовой формат. Пусть первое значение в столбце будет 0, второе 1:

for col in ['steering', 'docs', 'vendor']:
    test[col] = (test[col] == test[col].unique()[1]).astype('int')

In [ ]:
# Так-же можно сразу преобразовать признак с объемом двигателя в числовой. Опытным путем я выяснил что в
# датасете у некоторых записей отсутствует число (вероятно это электромобили), им поставим объем 0.0:

test.engine_volume = test.engine_volume.apply(lambda x: '0.0' if x == ' LTR' 
                                              else x.split()[0]).astype('float')

In [ ]:
# Можно столбец engine_power преобразовать в числовой:

test.engine_power = test.engine_power.apply(lambda x: x.split()[0]).astype('int')